# Import packages

In [1]:
import os
import json
import numpy as np
from sympy import symbols, sympify, lambdify

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.constraints import NonNeg

# Set random seeds for reproducibility
seed_value = 2023
tf.random.set_seed(seed_value)
np.random.seed(seed_value)

2023-11-09 12:37:04.612962: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-09 12:37:04.615296: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-09 12:37:04.657613: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-09 12:37:04.657653: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-09 12:37:04.657681: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

# Constants and parameters

In [2]:
# EQS_FILE = 'ERK_equations_s2p2.json'
# EQS_FILE = 'ERK_equations_s4p4.json'
EQS_FILE = 'vicky_case2.json'

EQS_PATH = os.path.join(os.getcwd(), 'RK_rootedtrees', EQS_FILE)


training_steps = 2001  #   5000 + 1
display_step = training_steps // 10

# learning_rate = 1e-2
learning_rate = tf.keras.optimizers.schedules.PiecewiseConstantDecay([training_steps // 3, 2 * training_steps // 3],[1e-2,5e-3,1e-3])

# Load Equations

In [3]:
# Read the JSON data from the file
with open(EQS_PATH, 'r') as file:
    data = json.load(file)

variables = data['variables']
equations = data['equations']
equation_terms = data['equation_terms']

# display(
#     f'variables = {variables}',
#     f'equations = {equations}',
#     f'equation_terms = {equation_terms}'
# )

print(f'equation_terms = {equation_terms}')

# Initialize a list to store the coefficients
scalar_coefficients = []
functions = []
bias_coefficients = []

# Loop through the equation terms
for terms in equation_terms:
    bias_coefficients.append(float(terms[0]))
    coefficients = []   
    for term in terms[1:]:
        # Split each term by the first '*'
        term_parts = term.split('*')
        # Extract the scalar coefficient or default to '1'
        scalar_coeff = term_parts[0] if len(term_parts) > 1 and term_parts[0] not in variables else '1'
        coefficients.append(float(scalar_coeff))
               
        # Remove first occurrence of scalar coefficient from the term
        func_coeff = term.replace(term_parts[0]+'*', '', 1) if len(term_parts) > 1 else term
        functions.append(func_coeff)
        
    scalar_coefficients.append(coefficients)   
        
print(bias_coefficients)
print(scalar_coefficients)
print(functions)

equation_terms = [['-1', 'a1', 'a2', 'a3', 'a4'], ['-1', 'b1', 'b2', 'b3', 'b4'], ['-1', 'c1', 'c2', 'c3', 'c4'], ['-1', 'd1', 'd2', 'd3', 'd4'], ['-0.5', 'a2*b1', 'a3*(b1+b2)', 'a4*(b1+b2+b3)'], ['-0.5', 'a2*c1', 'a3*(c1+c2)', 'a4*(c1+c2+c3)'], ['-0.5', 'a2*d1', 'a3*(d1+d2)', 'a4*(d1+d2+d3)'], ['-0.5', 'b2*c1', 'b3*(c1+c2)', 'b4*(c1+c2+c3)'], ['-0.5', 'b2*d1', 'b3*(d1+d2)', 'b4*(d1+d2+d3)'], ['-0.5', 'c2*d1', 'c3*(d1+d2)', 'c4*(d1+d2+d3)']]
[-1.0, -1.0, -1.0, -1.0, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5]
[[1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0, 1.0]]
['a1', 'a2', 'a3', 'a4', 'b1', 'b2', 'b3', 'b4', 'c1', 'c2', 'c3', 'c4', 'd1', 'd2', 'd3', 'd4', 'b1', '(b1+b2)', '(b1+b2+b3)', 'c1', '(c1+c2)', '(c1+c2+c3)', 'd1', '(d1+d2)', '(d1+d2+d3)', 'c1', '(c1+c2)', '(c1+c2+c3)', 'd1', '(d1+d2)', '(d1+d2+d3)', 'd1', '(d1+d2)', '(d1+d2+d3)']


# Model

## Create model

In [4]:
# Network Parameters
num_input = 1 # input layer

num_vars = len(variables)
num_eqs = len(equations)
num_functions = len(functions)

num_hidden = [num_vars, num_functions]
num_output = num_eqs # output layer

display(
    f'num_hidden = {num_hidden}',
    f'num_output = {num_output}'
)

'num_hidden = [16, 34]'

'num_output = 10'

In [5]:
## Weights to Layer 3 - determined by the functions contains the variables
w23_flags = [[True] * num_functions for _ in range(num_vars)]
for vi, var in enumerate(variables):
    for fi, func in enumerate(functions):
        # print(f'var = {var}, func = {func}', var not in func)
        if var not in func:
            w23_flags[vi][fi] = False

w23_flags = tf.constant(w23_flags, dtype=tf.bool)

# Initialize the weights (w23) with zeros
w23 = tf.constant(tf.zeros(num_hidden, dtype=tf.float32))
# Set the weights to 1 where func_flags is True
w23 = tf.where(w23_flags, tf.ones_like(w23), w23)
w23 = tf.transpose(w23)

## Weights to Layer 4 - determined by the coefficients of the functions
# Initialize the weights (w34) with zeros
w34_np = np.zeros([num_functions, num_output], dtype=np.float32)
# Assign scalar coefficients to the first column of w34_np
row = 0
for sci, scalar_coeffs in enumerate(scalar_coefficients):
    for scj, scl_coeff in enumerate(scalar_coeffs):
        w34_np[row, sci] = scl_coeff
        row += 1
# Tranform to tensor
w34 = tf.constant(w34_np, dtype=tf.float32)

## IC for classic methods
# w12_rk2 = tf.Variable([[0.8, 0.3, 0.3]])
# w12_rk4 = tf.Variable([[0.3, -0.1, 0.63, 0.2, 0.1, 0.8, 0.1, 0.2, 0.4, 0.1]])
w12_vicky = tf.Variable([[0.0, 0.0, 0.0, 1.5, 0.0, 0.0, 0.8, 0.8, 0.0, 0.8, 0.0, 0.9, 0.2, 0.0, 0.0, 1.0]],
                dtype=tf.float32, 
                constraint=NonNeg())

# Store layers weight & bias
weights = {
    ## Variables - weights to Layer 1
    # Create the variable with the non-negativity constraint
    # 'w12': tf.Variable(tf.random.normal([num_input, num_hidden[0]]), 
    #               dtype=tf.float32, 
    #               constraint=NonNeg()),
    # 'w12': w12_rk2,
    # 'w12': w12_rk4,
    'w12': w12_vicky,
    # Whether the functions in F1 and F2 contain the variables x1 and x2
    'w23': w23,
    # # The coefficients of the functions in F1 and F2
    'w34': w34
}

biases = {
    'b12': tf.constant(tf.zeros([num_hidden[0]], dtype=tf.float32)),
    'b23': tf.constant(tf.zeros([num_hidden[1]], dtype=tf.float32)),
    'out': tf.constant([bias_coefficients], dtype=tf.float32),
}

# Stochastic gradient descent optimizer.
optimizer = Adam(learning_rate=learning_rate, name='custom_optimizer_name')

## Model functions

In [6]:
variables_str = ' '.join(variables)
variables_sym = symbols(variables_str)
functions_sym = [sympify(func) for func in functions]   


def evaluate_tf_function(inputs, values, symbolic_function):
    # Ensure that the number of inputs and values match
    if len(inputs) != len(values):
        raise ValueError("Number of inputs and values must match.")

    # Evaluate the symbolic function using TensorFlow and the provided values
    result = symbolic_function(*values)
    # Convert the result to a TensorFlow tensor
    result = tf.convert_to_tensor(result, dtype=tf.float32)
    
    # print(result.__class__)
    
    return result

# Define the custom activation function with @tf.function
# @tf.function
def activation_layer2(layer, vars=variables_sym, funcs=functions_sym):
    
    layer_values = layer.numpy()[0]
    # var_vals = [layer_values[i] for i in range(layer_values.shape[0])]
    ##x1_val = tf.squeeze(layer[0, 0])
    var_vals = [tf.squeeze(layer[0, i]) for i in range(layer_values.shape[0])]
    
    # print(f'var_vals = {var_vals}')
   
    # var_vals_dict = dict(zip(vars, var_vals))
    # print(f'var_vals_dict = {var_vals_dict}')
    # # Step 3 and 4: Substitute values into the functions and evaluate
    # layer_act = [func.subs(var_vals_dict).evalf() for func in funcs]
    # # Convert SymPy Float to Python float
    # layer_act = [tf.convert_to_tensor(float(val.evalf()), dtype=tf.float32) for val in layer_act]
    
    layer_act = list()
    for func in funcs:
        func_tf = lambdify(vars, func, 'tensorflow')
        layer_act.append(evaluate_tf_function(variables_sym, var_vals, func_tf))
        # result = func_tf(*var_vals)
        # layer_act.append(result)
    
    # print(f'layer_act = {layer_act}')
    
    # layer_2 = tf.stack(layer_act, axis=0)  # Stack the list of tensors into a single tensor
    # layer_2 = tf.reshape(layer_2, [1, len(funcs)])
    
    # Convert layer_act to a TensorFlow tensor
    layer_2 = tf.convert_to_tensor(layer_act, dtype=tf.float32)
    
    # print(type(layer_2))
    
    # print(f'layer_2 = {layer_2}')
    
    layer_2 = tf.reshape(layer_2, [1, len(funcs)])

    return layer_2


# Create model
def multilayer_perceptron(x, weights, biases):

    # Reshape input if necessary, matching the shape of the first layer's weights
    x = tf.reshape(x, [1, -1])  # Adjust the shape as needed

    layer_1 = tf.add(tf.matmul(x, weights['w12']), biases['b12'])
    
    # print(f'layer_1 = {layer_1.shape}')

    layer_2 = activation_layer2(layer_1)
    
    # Output fully connected layer
    output = tf.add(tf.matmul(layer_2, weights['w34']), biases['out'])
    
    return output, layer_1


def loss_function(weights, biases):
    
    output, _= multilayer_perceptron(tf.constant(1.0, dtype=tf.float32), weights, biases)

    return tf.reduce_mean(tf.square(output))


# Train step
def train_step(weights, biases, optimizer):

    with tf.GradientTape() as tape:
        
        loss = loss_function(weights, biases)

    trainable_variables = [weights['w12']]  # list containing only 'w12'
    
    gradients = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))

    return loss    

## Train model

In [7]:
for i in range(training_steps):
       
    current_loss = train_step(weights, biases, optimizer)
    if i % display_step == 0:
        print(f"epoch {i} => loss: {current_loss:.16e} ")

epoch 0 => loss: 3.6800003051757812e-01 
epoch 500 => loss: 2.8421709853920481e-15 


In [ ]:
FUNC_RES, solution  = multilayer_perceptron(tf.constant(1.0, dtype=tf.float32), weights, biases)

print('SOLUTION')
for i, var in enumerate(variables):
    print(f'{var} = {solution[0, i]:.10f}')
    
print()
print('RESIDUALS')
for f, func in enumerate(equations):
    print(f'{func} = {FUNC_RES[0, f]:.4e}')

SOLUTION
a1 = 0.2979925275
a2 = 0.0023647952
a3 = 0.0023647952
a4 = 0.6972779036
b1 = 0.0200530048
b2 = 0.0963397101
b3 = 0.2471616268
b4 = 0.6364454627
c1 = 0.0000000000
c2 = 0.0000000000
c3 = 0.5000002980
c4 = 0.4999996722
d1 = 0.0802226365
d2 = 0.0000091307
d3 = 0.2593138814
d4 = 0.6604542732

RESIDUALS
a1 + a2 + a3 + a4 - 1 = 0.0000e+00
b1 + b2 + b3 + b4 - 1 = -1.7881e-07
c1 + c2 + c3 + c4 - 1 = 0.0000e+00
d1 + d2 + d3 + d4 - 1 = -5.9605e-08
a2*b1 + a3*(b1+b2) + a4(b1+b2+b3) - 0.5 = 5.9605e-08
a2*c1 + a3*(c1+c2) + a4(c1+c2+c3) - 0.5 = 2.9802e-07
a2*d1 + a3*(d1+d2) + a4(d1+d2+d3) - 0.5 = 5.9605e-08
b2*c1 + b3*(c1+c2) + b4(c1+c2+c3) - 0.5 = 2.9802e-07
b2*d1 + b3*(d1+d2) + b4(d1+d2+d3) - 0.5 = 5.9605e-08
c2*d1 + c3*(d1+d2) + c4(d1+d2+d3) - 0.5 = 5.9605e-08
